In [21]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage import feature
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
%matplotlib inline

In [22]:
#To return LBP histogram gradient
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
    # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

In [23]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [24]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,35,55,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female   male
age                    
(0, 35]    10440  13590
(35, 55]    3930   3600
(55, 99]    1230    870


In [25]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
age_label_18_35 = []
lbp_features_18_35 = []
age_label_36_55 = []
lbp_features_36_55 = []
age_label_56_ = []
lbp_features_56_ = []


# initialize the local binary patterns descriptor along with
desc = LocalBinaryPatterns(24, 8)

with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    print(len(image_file_name))
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #convert the images to gray and describe it to extract lbp features
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        hist = desc.describe(gray)
        
        if age[i] >= 10 and age[i]<=35:
            lbp_features_18_35.append(hist)
            if gender[i]== 'male':
                age_label_18_35.append("male")
            else:
                age_label_18_35.append("female")
        elif age[i]>36 and age[i] <=55:    
            lbp_features_36_55.append(hist)
            if gender[i]== 'male':
                age_label_36_55.append("male")
            else:
                age_label_36_55.append("female")
        elif age[i]>55:    
            lbp_features_56_.append(hist)
            if gender[i]== 'male':
                age_label_56_.append("male")
            else:
                age_label_56_.append("female") 
                
            

33660


# Age Range 18-35

In [26]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_18_35)
X =  np.array(lbp_features_18_35)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 18-35")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 18-35
Overall Accuracy using Linear SVM: 0.6211402413649605
              precision    recall  f1-score   support

      female       0.64      0.29      0.40      5235
        male       0.62      0.88      0.72      6780

    accuracy                           0.62     12015
   macro avg       0.63      0.58      0.56     12015
weighted avg       0.63      0.62      0.58     12015

Accuracy for each class
[0.29092646 0.87610619]




Non-Linear RBF Accuracy: 0.5787765293383271
              precision    recall  f1-score   support

      female       0.67      0.07      0.12      5235
        male       0.57      0.97      0.72      6780

    accuracy                           0.58     12015
   macro avg       0.62      0.52      0.42     12015
weighted avg       0.62      0.58      0.46     12015

              precision    recall  f1-score   support

      female       0.67      0.07      0.12      5235
        male       0.57      0.97      0.72      6780

    accuracy    

Iteration 194, loss = 0.67962061
Iteration 195, loss = 0.67955679
Iteration 196, loss = 0.67953670
Iteration 197, loss = 0.67952896
Iteration 198, loss = 0.67947261
Iteration 199, loss = 0.67947557
Iteration 200, loss = 0.67949574
Iteration 201, loss = 0.67942707
Iteration 202, loss = 0.67943762
Iteration 203, loss = 0.67938893
Iteration 204, loss = 0.67937101
Iteration 205, loss = 0.67934167
Iteration 206, loss = 0.67927612
Iteration 207, loss = 0.67931353
Iteration 208, loss = 0.67926311
Iteration 209, loss = 0.67923649
Iteration 210, loss = 0.67922039
Iteration 211, loss = 0.67918210
Iteration 212, loss = 0.67918257
Iteration 213, loss = 0.67913485
Iteration 214, loss = 0.67914408
Iteration 215, loss = 0.67910933
Iteration 216, loss = 0.67910241
Iteration 217, loss = 0.67905781
Iteration 218, loss = 0.67909225
Iteration 219, loss = 0.67906007
Iteration 220, loss = 0.67901129
Iteration 221, loss = 0.67900585
Iteration 222, loss = 0.67896362
Iteration 223, loss = 0.67894593
Iteration 

Iteration 444, loss = 0.67365464
Iteration 445, loss = 0.67363810
Iteration 446, loss = 0.67363173
Iteration 447, loss = 0.67363949
Iteration 448, loss = 0.67356498
Iteration 449, loss = 0.67346012
Iteration 450, loss = 0.67342773
Iteration 451, loss = 0.67337677
Iteration 452, loss = 0.67330665
Iteration 453, loss = 0.67343392
Iteration 454, loss = 0.67343458
Iteration 455, loss = 0.67324678
Iteration 456, loss = 0.67322231
Iteration 457, loss = 0.67316973
Iteration 458, loss = 0.67314803
Iteration 459, loss = 0.67307066
Iteration 460, loss = 0.67314509
Iteration 461, loss = 0.67295628
Iteration 462, loss = 0.67294980
Iteration 463, loss = 0.67294682
Iteration 464, loss = 0.67276022
Iteration 465, loss = 0.67284834
Iteration 466, loss = 0.67273553
Iteration 467, loss = 0.67273076
Iteration 468, loss = 0.67263362
Iteration 469, loss = 0.67257683
Iteration 470, loss = 0.67249657
Iteration 471, loss = 0.67250699
Iteration 472, loss = 0.67241663
Iteration 473, loss = 0.67234750
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Age Range 36-55

In [27]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_36_55)
X =  np.array(lbp_features_36_55)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 36-55")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 36-55
Overall Accuracy using Linear SVM: 0.5738888888888889
              precision    recall  f1-score   support

      female       0.59      0.63      0.61      1890
        male       0.56      0.51      0.53      1710

    accuracy                           0.57      3600
   macro avg       0.57      0.57      0.57      3600
weighted avg       0.57      0.57      0.57      3600

Accuracy for each class
[0.63280423 0.50877193]




Non-Linear RBF Accuracy: 0.5419444444444445
              precision    recall  f1-score   support

      female       0.55      0.77      0.64      1890
        male       0.53      0.29      0.38      1710

    accuracy                           0.54      3600
   macro avg       0.54      0.53      0.51      3600
weighted avg       0.54      0.54      0.51      3600

              precision    recall  f1-score   support

      female       0.55      0.77      0.64      1890
        male       0.53      0.29      0.38      1710

    accuracy    

Iteration 194, loss = 0.69036922
Iteration 195, loss = 0.69036572
Iteration 196, loss = 0.69035405
Iteration 197, loss = 0.69034609
Iteration 198, loss = 0.69033838
Iteration 199, loss = 0.69033706
Iteration 200, loss = 0.69032545
Iteration 201, loss = 0.69031144
Iteration 202, loss = 0.69032139
Iteration 203, loss = 0.69028826
Iteration 204, loss = 0.69029122
Iteration 205, loss = 0.69030249
Iteration 206, loss = 0.69026992
Iteration 207, loss = 0.69031230
Iteration 208, loss = 0.69025294
Iteration 209, loss = 0.69024510
Iteration 210, loss = 0.69025178
Iteration 211, loss = 0.69024077
Iteration 212, loss = 0.69023028
Iteration 213, loss = 0.69020977
Iteration 214, loss = 0.69020525
Iteration 215, loss = 0.69020720
Iteration 216, loss = 0.69020138
Iteration 217, loss = 0.69018303
Iteration 218, loss = 0.69017669
Iteration 219, loss = 0.69017061
Iteration 220, loss = 0.69017278
Iteration 221, loss = 0.69014803
Iteration 222, loss = 0.69013873
Iteration 223, loss = 0.69013686
Iteration 

Iteration 448, loss = 0.68788781
Iteration 449, loss = 0.68785110
Iteration 450, loss = 0.68784039
Iteration 451, loss = 0.68782409
Iteration 452, loss = 0.68782563
Iteration 453, loss = 0.68781413
Iteration 454, loss = 0.68785813
Iteration 455, loss = 0.68780506
Iteration 456, loss = 0.68775960
Iteration 457, loss = 0.68774280
Iteration 458, loss = 0.68773408
Iteration 459, loss = 0.68774322
Iteration 460, loss = 0.68772813
Iteration 461, loss = 0.68771955
Iteration 462, loss = 0.68769323
Iteration 463, loss = 0.68765811
Iteration 464, loss = 0.68763289
Iteration 465, loss = 0.68762496
Iteration 466, loss = 0.68760545
Iteration 467, loss = 0.68760269
Iteration 468, loss = 0.68759141
Iteration 469, loss = 0.68759296
Iteration 470, loss = 0.68759881
Iteration 471, loss = 0.68756427
Iteration 472, loss = 0.68754169
Iteration 473, loss = 0.68752456
Iteration 474, loss = 0.68756035
Iteration 475, loss = 0.68751813
Iteration 476, loss = 0.68749167
Iteration 477, loss = 0.68748240
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Age Range >55

In [28]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_56_)
X =  np.array(lbp_features_56_)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range > 55")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range > 55
Overall Accuracy using Linear SVM: 0.6114285714285714
              precision    recall  f1-score   support

      female       0.62      0.88      0.73       619
        male       0.57      0.22      0.32       431

    accuracy                           0.61      1050
   macro avg       0.59      0.55      0.52      1050
weighted avg       0.60      0.61      0.56      1050

Accuracy for each class
[0.88206785 0.22273782]




Non-Linear RBF Accuracy: 0.5895238095238096
              precision    recall  f1-score   support

      female       0.59      1.00      0.74       619
        male       0.00      0.00      0.00       431

    accuracy                           0.59      1050
   macro avg       0.29      0.50      0.37      1050
weighted avg       0.35      0.59      0.44      1050

              precision    recall  f1-score   support

      female       0.59      1.00      0.74       619
        male       0.00      0.00      0.00       431

    accuracy     

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.6104761904761905
              precision    recall  f1-score   support

      female       0.62      0.89      0.73       619
        male       0.57      0.20      0.30       431

    accuracy                           0.61      1050
   macro avg       0.59      0.55      0.51      1050
weighted avg       0.60      0.61      0.55      1050

              precision    recall  f1-score   support

      female       0.62      0.89      0.73       619
        male       0.57      0.20      0.30       431

    accuracy                           0.61      1050
   macro avg       0.59      0.55      0.51      1050
weighted avg       0.60      0.61      0.55      1050

Accuracy for each class
[0.89499192 0.20185615]




Iteration 1, loss = 0.68089823
Iteration 2, loss = 0.68087519
Iteration 3, loss = 0.68080652
Iteration 4, loss = 0.68074294
Iteration 5, loss = 0.68068870
Iteration 6, loss = 0.68060793
Iteration 7, loss = 0.68054392
Iteration 8, loss = 0.68045677
I

Iteration 241, loss = 0.67645480
Iteration 242, loss = 0.67646354
Iteration 243, loss = 0.67645033
Iteration 244, loss = 0.67642764
Iteration 245, loss = 0.67646977
Iteration 246, loss = 0.67644058
Iteration 247, loss = 0.67640648
Iteration 248, loss = 0.67639624
Iteration 249, loss = 0.67638576
Iteration 250, loss = 0.67636735
Iteration 251, loss = 0.67632423
Iteration 252, loss = 0.67632062
Iteration 253, loss = 0.67630194
Iteration 254, loss = 0.67628509
Iteration 255, loss = 0.67625821
Iteration 256, loss = 0.67624899
Iteration 257, loss = 0.67621533
Iteration 258, loss = 0.67619854
Iteration 259, loss = 0.67619398
Iteration 260, loss = 0.67617267
Iteration 261, loss = 0.67616346
Iteration 262, loss = 0.67612817
Iteration 263, loss = 0.67612962
Iteration 264, loss = 0.67609219
Iteration 265, loss = 0.67606418
Iteration 266, loss = 0.67610639
Iteration 267, loss = 0.67608129
Iteration 268, loss = 0.67606475
Iteration 269, loss = 0.67604446
Iteration 270, loss = 0.67601051
Iteration 

Iteration 500, loss = 0.67108797
MLP Accuracy: 0.5895238095238096
              precision    recall  f1-score   support

      female       0.59      1.00      0.74       619
        male       0.00      0.00      0.00       431

    accuracy                           0.59      1050
   macro avg       0.29      0.50      0.37      1050
weighted avg       0.35      0.59      0.44      1050

Accuracy for each class
[1. 0.]






/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-